In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
import os

# --- Parameters ---
dataset_path = "/content/drive/MyDrive/SignLanguageDataset/dataset"
img_size = (64, 64)
batch_size = 32

# --- 1. Load the Dataset ---
# We load the raw data, the model will handle normalization.
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=42
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=42
)

# Get and save class names
class_names = train_ds.class_names
print("Detected classes:", class_names)
with open("class_names.json", "w") as f:
    json.dump(class_names, f)

# Improve performance with cache and prefetch
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# --- 2. Define the Simple CNN Model (Corrected for TF.js Compatibility) ---

# We remove the separate keras.Input layer and add the 'input_shape'
# argument directly to the first layer (the Rescaling layer).

model = keras.models.Sequential([
    # Add input_shape to the very first layer
    keras.layers.Rescaling(1./255, input_shape=(img_size[0], img_size[1], 3)),

    # The rest of the model remains the same
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(class_names), activation='softmax')
])

# --- 3. Compile the Model ---
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


# --- 4. Train the Model with Callbacks ---
# Define callbacks for smarter training
callbacks = [
    # Save only the best performing model
    tf.keras.callbacks.ModelCheckpoint(
        "best_model.keras",
        save_best_only=True,
        monitor="val_accuracy"
    ),
    # Stop training if there's no improvement
    tf.keras.callbacks.EarlyStopping(
        patience=3,
        monitor="val_loss"
    )
]

# Start training
print("\n🚀 Starting training with the simple model...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks,
    verbose=1 # Show progress bar
)
print("\n✅ Training complete.")



Found 1200 files belonging to 4 classes.
Using 960 files for training.
Found 1200 files belonging to 4 classes.
Using 240 files for validation.
Detected classes: ['A', 'B', 'C', 'D']


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_4 (Rescaling)         │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,716 (2.61 MB)

 Trainable params: 683,716 (2.61 MB)

 Non-trainable params: 0 (0.00 B)


🚀 Starting training with the simple model...
Epoch 1/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 15s 330ms/step - accuracy: 0.6004 - loss: 0.9332 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 2/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 227ms/step - accuracy: 0.9989 - loss: 0.0035 - val_accuracy: 0.9958 - val_loss: 0.0067
Epoch 3/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 201ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 1.0000 - val_loss: 1.1127e-04
Epoch 4/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step - accuracy: 1.0000 - loss: 2.8079e-05 - val_accuracy: 1.0000 - val_loss: 2.9330e-05
Epoch 5/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 242ms/step - accuracy: 1.0000 - loss: 1.5960e-05 - val_accuracy: 1.0000 - val_loss: 2.1125e-05
Epoch 6/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 242ms/step - accuracy: 1.0000 - loss: 1.5201e-05 - val_accuracy: 1.0000 - val_loss: 1.7548e-05
Epoch 7/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 239ms/step - accuracy: 1.0000 - loss: 1.0720e-05 - val_accuracy: 1.0000 - val_loss: 1.4333e-05
Epoch 8/15
30/30 ━━━━

In [ ]:
# --- 5. Inference Section ---
print("\nLoading best model for inference...")
# Load the best model saved by the callback
best_model = tf.keras.models.load_model("best_model.keras")

# Load class names
with open("class_names.json", "r") as f:
    class_names = json.load(f)

# Define the prediction function
def predict_image(img_path):
    img = tf.keras.utils.load_img(img_path, target_size=img_size)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create batch dimension

    # NO manual normalization needed here, the model handles it!

    predictions = best_model.predict(img_array, verbose=0)
    predicted_class = class_names[np.argmax(predictions)]
    confidence = np.max(predictions)

    print(f"\nImage: {img_path}")
    print(f"Predicted Class: {predicted_class} (Confidence: {confidence:.2f})")

# Example usage (change path to your test image)
predict_image("/content/drive/MyDrive/SignLanguageDataset/test/rightD.png")


Loading best model for inference...

Image: /content/drive/MyDrive/SignLanguageDataset/test/rightD.png
Predicted Class: D (Confidence: 0.69)


In [ ]:
!pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2025.7.1 requires packaging>=24.1, but you have packaging 23.2 which is incompatible.
google-cloud-bigquery 3.35.1 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [ ]:
import tensorflow as tf
import tensorflowjs as tfjs
import os

# Define the path to your saved Keras model and the desired output path
keras_model_path = "best_model.keras"
tfjs_model_path = "model_web"

# Create the output directory if it doesn't exist
os.makedirs(tfjs_model_path, exist_ok=True)

# Load your Keras model
print(f"Loading Keras model from: {keras_model_path}")
model = tf.keras.models.load_model(keras_model_path)

# Convert and save the model to TensorFlow.js format
print(f"Converting model and saving to: {tfjs_model_path}")
tfjs.converters.save_keras_model(model, tfjs_model_path)

print("\n✅ Conversion complete!")
print(f"Your TensorFlow.js model is now in the '{tfjs_model_path}/' directory.")

Loading Keras model from: best_model.keras
Converting model and saving to: model_web
failed to lookup keras version from the file,
    this is likely a weight only file

✅ Conversion complete!
Your TensorFlow.js model is now in the 'model_web/' directory.


In [ ]:
# Zip the folder for easy download
!zip -r model_web.zip ./model_web

print("\n✅ 'model_web.zip' created. You can now download it from the file browser on the left.")


  adding: model_web/ (stored 0%)
  adding: model_web/model.json (deflated 84%)
  adding: model_web/group1-shard1of1.bin (deflated 8%)

✅ 'model_web.zip' created. You can now download it from the file browser on the left.
